In [1]:
import numpy as np
import pandas as pd
import argparse
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn import tree
from sklearn.linear_model import LinearRegression
from sklearn.tree import DecisionTreeRegressor
from pandas.tseries.offsets import DateOffset 
from sklearn.preprocessing import MinMaxScaler
from lineartree import LinearTreeRegressor
from joblib import dump, load
import requests
import xmltodict
from dateutil.relativedelta import relativedelta
import warnings
import json
#warnings.filterwarnings('ignore')

In [2]:
last_know_date = '2023-03-31'

products_arbitrable = {
    'DAV_STABLE': False,
    'DAV_VOL': True,
    'DAT': True,
    'PEL': True,
    'Livrets': True,
    'LA_LDD': True,
    'Autres': False,
    'AV_EURO': True,
    'AV_UC': True
}

products_is_liquid = {
    'DAV_STABLE': True,
    'DAV_VOL':  True,
    'DAT': False,
    'PEL': False,
    'Livrets': True,
    'LA_LDD': True,
    'Autres': False,
    'AV_EURO': False,
    'AV_UC': False,
}

quantile = 0.75


# I. Loading data from Banque de France. INSEE and Economic Scenario

**I.1 BdF**

In [16]:
#REQUETE des données WEBSTAT en trimestriel

session = requests.Session()
liste_trimestrielle = [
    'CFT.Q.N.FR.W0.S1M.S1.N.A.LE.F62A._Z._Z.XDC._T.S.V.N._T',
    'CFT.Q.N.FR.W0.S1M.S1.N.A.LE.F62B._Z._Z.XDC._T.S.V.N._T',
    'CNF.Q.N.FR.W0.S1M.S1.N.A.LE.F2.T._Z.XDC._T.S.V.N._T',
    'CNF.Q.N.FR.W0.S1M.S1.N.L.F.F8.T._Z.XDC._T.S.V.N._T',
    'CNF.Q.N.FR.W0.S1M.S1.N.A.F.F8.T._Z.XDC._T.S.V.N._T',
    'CNF.Q.N.FR.W0.S1M.S1.N.L.F.F4.T._Z.XDC._T.S.V.N._T',
    'CNF.Q.N.FR.W0.S1M.S1.N.A.F.F4.T._Z.XDC._T.S.V.N._T',
    'CNF.Q.N.FR.W0.S1M.S1.N.A.F.F3.T._Z.XDC._T.S.V.N._T',
    'CNF.Q.N.FR.W0.S1M.S1.N.A.F.F5._Z._Z.XDC._T.S.V.N._T',
    'CNF.Q.N.FR.W0.S1M.S1.N.L.F.F5._Z._Z.XDC._T.S.V.N._T',
    'CNF.Q.N.FR.W0.S11.S1.N.L.F.F3.T._Z.XDC._T.S.V.N._T',
    'CNF.Q.N.FR.W0.S11.S1.N.A.F.F3.T._Z.XDC._T.S.V.N._T',
    'CNF.Q.N.FR.W0.S11.S1.N.L.F.F5._Z._Z.XDC._T.S.V.N._T',
    'CNF.Q.N.FR.W0.S11.S1.N.A.F.F51._Z._Z.XDC._T.S.V.N._T',
    'CNF.Q.N.FR.W0.S11.S1.N.A.LE.F8.T._Z.XDC._T.S.V.N._T',
    'CNF.Q.N.FR.W0.S11.S1.N.L.LE.F8.T._Z.XDC._T.S.V.N._T',
    'CNF.Q.N.FR.W0.S11.S1.N.A.F.F6._Z._Z.XDC._T.S.V.N._T',
    'CNF.Q.N.FR.W0.S11.S1.N.L.F.F6._Z._Z.XDC._T.S.V.N._T',
    'CNF.Q.N.FR.W0.S11.S1.N.A.LE.F521._Z._Z.XDC._T.S.V.N._T',
    'CNF.Q.N.FR.W0.S14.S1.N.A.LE.F2.T._Z.XDC._T.S.V.N._T']



headers = {
    'Authorization' : "Apikey c89fc7a5c0c717b5aa3256585b885e593ae750d133809c9ccf85a373", # à remplacer par votre clef d'api récupéré sur le site 
    'accept': "application/json"
}

url0 = "https://webstat.banque-france.fr/api/explore/v2.1/catalog/datasets/observations/exports/json/?where=series_key+IN+%28%22"

df_quater = pd.DataFrame()

for id in liste_trimestrielle:
    url = url0 + id + "%22%29&order_by=-time_period_start"

    # URL de l'exemple 2 (observations d'un groupe de séries à une date donnée) 
    result = session.get(url, verify=False, headers=headers) 
    datajson = result.json()

    # Pretty print du JSON
    pretty_json = json.dumps(datajson, indent=4)
    data = pretty_json

    # Si data est une chaîne de caractères JSON, on la convertit en liste de dictionnaires
    if isinstance(data, str):
        try:
            data = json.loads(data)
        except json.JSONDecodeError as e:
            print(f"Erreur lors de la conversion JSON: {e}")

    # Vérification que data est bien une liste de dictionnaires
    if isinstance(data, list) and all(isinstance(item, dict) for item in data):
        try:
            # Création du DataFrame
            df = pd.DataFrame({
                'Date': [item['time_period_end'] for item in data],
                'Titre_FR': [item['title_fr'] for item in data],
                'Valeur': [item['obs_value'] for item in data]
            })

            # Affichage du DataFrame
        except KeyError as e:
            print(f"Clé manquante: {e}")

    df_quater = pd.concat([df_quater, df], ignore_index=True)

pd.set_option('display.max_rows', None)

df_quater['Date'] = pd.to_datetime(df_quater['Date'])

# Remplacer les valeurs NA dans la colonne "Valeur" par 0
df_quater['Valeur'] = df_quater['Valeur'].fillna(0)

# Pivot du DataFrame pour transformer les "Titre_FR" en colonnes et "Date_fin" en index
df_pivoted = df_quater.pivot(index='Date', columns='Titre_FR', values='Valeur')

# Réinitialiser l'index pour remettre "Date_fin" en colonne
df_pivoted = df_pivoted.reset_index()
df_bdf_quarterly = df_pivoted.sort_values(by='Date', ascending=False)


for col in df_bdf_quarterly.columns:
    if col != 'Date':
        df_bdf_quarterly[col] = df_bdf_quarterly[col] * 1e3

# Afficher le DataFrame transformé
display(df_bdf_quarterly)


C:\Users\admin\Anaconda\anaconda3\Lib\site-packages\urllib3\connectionpool.py:1056: InsecureRequestWarning: Unverified HTTPS request is being made to host 'webstat.banque-france.fr'. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/1.26.x/advanced-usage.html#ssl-warnings
  warnings.warn(
C:\Users\admin\Anaconda\anaconda3\Lib\site-packages\urllib3\connectionpool.py:1056: InsecureRequestWarning: Unverified HTTPS request is being made to host 'webstat.banque-france.fr'. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/1.26.x/advanced-usage.html#ssl-warnings
  warnings.warn(
C:\Users\admin\Anaconda\anaconda3\Lib\site-packages\urllib3\connectionpool.py:1056: InsecureRequestWarning: Unverified HTTPS request is being made to host 'webstat.banque-france.fr'. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/1.26.x/advanced-usage.html#ssl-warnings
  warnings.warn(
C:\U

Titre_FR,Date,Actif Toutes dénominations monétaires des SNF vis-à-vis de toutes contreparties - encours d'OPC monétaires,Actif Toutes dénominations monétaires des SNF vis-à-vis de toutes contreparties - flux d'actions et titres d'OPC,Actif Toutes dénominations monétaires des SNF vis-à-vis de toutes contreparties - flux de police d'assurance et fonds de pension,Actif Toutes dénominations monétaires des SNF vis-à-vis de toutes contreparties - flux de titres de créances,Actif Toutes dénominations monétaires des ménages vis-à-vis de toutes contreparties - encours de monnaie et dépôts,Actif Toutes dénominations monétaires des ménages vis-à-vis de toutes contreparties - flux d'actions et titres d'OPC,Actif Toutes dénominations monétaires des ménages vis-à-vis de toutes contreparties - flux de crédits,Actif Toutes dénominations monétaires des ménages vis-à-vis de toutes contreparties - flux de titres de créances,Actif Toutes dénominations monétaires des ménages vis-à-vis de toutes contreparties - flux des autres comptes à recevoir ou à payer,"Assurance-vie en unités de compte, actif des ménages, encours trimestriel","Assurance-vie support euro, actif des ménages, encours trimestriel","Ménages et ISBLSM, Actif, Monnaie et dépôts",Passif Toutes dénominations monétaires des SNF vis-à-vis de toutes contreparties - flux d'actions et titres d'OPC,Passif Toutes dénominations monétaires des SNF vis-à-vis de toutes contreparties - flux de titres de créances,Passif Toutes dénominations monétaires des ménages vis-à-vis de toutes contreparties - flux d'actions et titres d'OPC,Passif Toutes dénominations monétaires des ménages vis-à-vis de toutes contreparties - flux de crédits,Passif Toutes dénominations monétaires des ménages vis-à-vis de toutes contreparties - flux des autres comptes à recevoir ou à payer,Passif des SNF vis-à-vis de toutes contreparties - flux de police d'assurance et fonds de pension,"Sociétés non financières, Actif, Autres comptes à recevoir / à payer","Sociétés non financières, Passif, Autres comptes à recevoir / à payer"
185,2024-03-31,116555000.0,-15821000.0,4065000.0,4625000.0,1.987605e+09,-5019000.0,93000.0,1658000.0,13723000.0,516600.0,1498488.0,2.083687e+09,23514000.0,-4120000.0,0.0,-5268000.0,-4908000.0,0.0,1.670124e+09,1.275308e+09
184,2023-12-31,95009000.0,51784000.0,-601000.0,-1991000.0,1.986409e+09,-1270000.0,1081000.0,1527000.0,-26891000.0,489101.0,1482693.0,2.082127e+09,44831000.0,4961000.0,0.0,1822000.0,1423000.0,0.0,1.581359e+09,1.260307e+09
183,2023-09-30,96868000.0,23079000.0,-978000.0,4094000.0,1.991095e+09,8511000.0,-791000.0,1927000.0,16049000.0,465300.0,1427255.0,2.084891e+09,20833000.0,6967000.0,0.0,3903000.0,-1458000.0,0.0,1.534746e+09,1.242780e+09
182,2023-06-30,89122000.0,37304000.0,217000.0,2886000.0,1.971170e+09,9718000.0,500000.0,1818000.0,20473000.0,472500.0,1453955.0,2.065474e+09,27373000.0,2721000.0,0.0,9682000.0,-16179000.0,0.0,1.528596e+09,1.209753e+09
181,2023-03-31,93754000.0,-18786000.0,3686000.0,2083000.0,1.952205e+09,1219000.0,-12000.0,577000.0,27610000.0,457000.0,1460436.0,2.046393e+09,9858000.0,-3476000.0,0.0,7872000.0,18326000.0,0.0,1.517714e+09,1.191769e+09
180,2022-12-31,75421000.0,17331000.0,-1617000.0,4975000.0,1.932922e+09,7316000.0,-977000.0,1382000.0,-30768000.0,432500.0,1451084.0,2.027017e+09,32964000.0,1248000.0,0.0,15422000.0,-13483000.0,0.0,1.507163e+09,1.189609e+09
179,2022-09-30,65343000.0,35016000.0,-165000.0,1218000.0,1.933260e+09,5741000.0,32000.0,2604000.0,3061000.0,440700.0,1435705.0,2.026928e+09,16868000.0,-2362000.0,0.0,22379000.0,-12255000.0,0.0,1.465127e+09,1.169398e+09
178,2022-06-30,62477000.0,-8690000.0,-1124000.0,7423000.0,1.900555e+09,24104000.0,675000.0,2418000.0,-237000.0,447699.0,1492032.0,1.995886e+09,27737000.0,-4190000.0,0.0,29823000.0,-10820000.0,0.0,1.443986e+09,1.133223e+09
177,2022-03-31,65619000.0,47302000.0,2588000.0,3031000.0,1.871204e+09,-943000.0,485000.0,-258000.0,48742000.0,473200.0,1598413.0,1.966340e+09,13775000.0,707000.0,0.0,16159000.0,

In [17]:
#REQUETE des données WEBSTAT en mensuel

session = requests.Session()
liste_mensuelle1 = [
    'BSI1.M.FR.N.A.L21.A.1.U6.2250.Z01.E',
    'BSI1.M.FR.N.A.L22.A.1.U6.2250.Z01.E',
    'BSI1.M.FR.N.A.L22FRPL.A.1.U6.2251.Z01.E',
    'BSI1.M.FR.N.A.L23.A.1.U6.2250.Z01.E',
    'BSI1.M.FR.N.A.L23FRLA.A.1.U6.2250.Z01.E',
    'BSI1.M.FR.N.A.L23FRLD.A.1.U6.2251.Z01.E',
    'BSI1.M.FR.N.R.A220Z.A.1.U6.2250.Z01.E',
    'BSI1.M.FR.N.R.A210Z.A.1.U6.2250.Z01.E',
    'BSI1.M.FR.N.R.A2N1Z.A.1.U6.2240.Z01.E',
    'BSI1.M.FR.N.R.A2N2Z.A.1.U6.2240.Z01.E',
    'BSI1.M.FR.N.F.L30.A.8.Z5.0000.Z01.T',
    'BSI1.M.FR.N.A.L22.A.1.U6.2240.Z01.E',
    'BSI1.M.FR.N.A.L21.A.1.U6.2240.Z01.E',
    'BSI1.M.FR.Y.R.A220Z.A.4.U6.2254FR.Z01.E',
    'BSI1.M.FR.N.R.A220Z.A.4.U6.2254FR.Z01.E',
]

liste_mensuelle2 = [
    'MIR.M.FR.B.L22.H.R.A.2240.EUR.N',
    'MIR1.M.FR.B.A22.A.5.A.2254U6.EUR.N',
    'MIR1.M.FR.B.A2B.A.5.A.2250U6.EUR.N',
    'MIR1.M.FR.B.A22HR.A.5.A.2254U6.EUR.N'
]


headers = {
    'Authorization' : "Apikey c89fc7a5c0c717b5aa3256585b885e593ae750d133809c9ccf85a373", # à remplacer par votre clef d'api récupéré sur le site 
    'accept': "application/json"
}

url0 = "https://webstat.banque-france.fr/api/explore/v2.1/catalog/datasets/observations/exports/json/?where=series_key+IN+%28%22"

df_mens2 = pd.DataFrame()

#liste mensuelle 2 avec multiplier par 1e^3

for id in liste_mensuelle2:
    url = url0 + id + "%22%29&order_by=-time_period_start"

    # URL de l'exemple 2 (observations d'un groupe de séries à une date donnée) 
    result = session.get(url, verify=False, headers=headers) 
    datajson = result.json()

    # Pretty print du JSON
    pretty_json = json.dumps(datajson, indent=4)
    data = pretty_json

    # Si data est une chaîne de caractères JSON, on la convertit en liste de dictionnaires
    if isinstance(data, str):
        try:
            data = json.loads(data)
        except json.JSONDecodeError as e:
            print(f"Erreur lors de la conversion JSON: {e}")

    # Vérification que data est bien une liste de dictionnaires
    if isinstance(data, list) and all(isinstance(item, dict) for item in data):
        try:
            # Création du DataFrame
            df = pd.DataFrame({
                'Date': [item['time_period_end'] for item in data],
                'Titre_FR': [item['title_fr'] for item in data],
                'Valeur': [item['obs_value'] for item in data]
            })

            # Affichage du DataFrame
        except KeyError as e:
            print(f"Clé manquante: {e}")

    df_mens2 = pd.concat([df_mens2, df], ignore_index=True)

pd.set_option('display.max_rows', None)

df_mens2['Date'] = pd.to_datetime(df_mens2['Date'])


# Pivot du DataFrame pour transformer les "Titre_FR" en colonnes et "Date_fin" en index
df_pivoted2 = df_mens2.pivot(index='Date', columns='Titre_FR', values='Valeur')

# Réinitialiser l'index pour remettre "Date_fin" en colonne
df_pivoted2 = df_pivoted2.reset_index()
df_bdf_mens2 = df_pivoted2.sort_values(by='Date', ascending=False)

for col in df_bdf_mens2.columns:
    if col != 'Date':
        df_bdf_mens2[col] = df_bdf_mens2[col] * 1e3

# Afficher le DataFrame transformé

#liste mensuelle 1 sans faire *10^3

df_mens1 = pd.DataFrame()

for id in liste_mensuelle1:
    url = url0 + id + "%22%29&order_by=-time_period_start"

    # URL de l'exemple 2 (observations d'un groupe de séries à une date donnée) 
    result = session.get(url, verify=False, headers=headers) 
    datajson = result.json()

    # Pretty print du JSON
    pretty_json = json.dumps(datajson, indent=4)
    data = pretty_json

    # Si data est une chaîne de caractères JSON, on la convertit en liste de dictionnaires
    if isinstance(data, str):
        try:
            data = json.loads(data)
        except json.JSONDecodeError as e:
            print(f"Erreur lors de la conversion JSON: {e}")

    # Vérification que data est bien une liste de dictionnaires
    if isinstance(data, list) and all(isinstance(item, dict) for item in data):
        try:
            # Création du DataFrame
            df = pd.DataFrame({
                'Date': [item['time_period_end'] for item in data],
                'Titre_FR': [item['title_fr'] for item in data],
                'Valeur': [item['obs_value'] for item in data]
            })

            # Affichage du DataFrame
        except KeyError as e:
            print(f"Clé manquante: {e}")

    df_mens1 = pd.concat([df_mens1, df], ignore_index=True)

pd.set_option('display.max_rows', None)

df_mens1['Date'] = pd.to_datetime(df_mens1['Date'])

# Pivot du DataFrame pour transformer les "Titre_FR" en colonnes et "Date_fin" en index
df_pivoted1 = df_mens1.pivot(index='Date', columns='Titre_FR', values='Valeur')

# Réinitialiser l'index pour remettre "Date_fin" en colonne
df_pivoted1 = df_pivoted1.reset_index()
df_bdf_mens1 = df_pivoted1.sort_values(by='Date', ascending=False)


# Afficher le DataFrame transformé

df_bdf_monthly = df_bdf_mens1.merge(df_bdf_mens2, on='Date', how='outer')

df_bdf_monthly[["Titres d'OPC monétaires, taux de valorisation de l'encours"]] = df_bdf_monthly[["Titres d'OPC monétaires, taux de valorisation de l'encours"]] / 100
df_bdf_monthly[["Taux d'intérêt sur contrats nouveaux, DAT, SNF, durée supérieure à 2 ans"]] = df_bdf_monthly[[ "Taux d'intérêt sur contrats nouveaux, DAT, SNF, durée supérieure à 2 ans"]] / 100000

display(df_bdf_monthly)

C:\Users\admin\Anaconda\anaconda3\Lib\site-packages\urllib3\connectionpool.py:1056: InsecureRequestWarning: Unverified HTTPS request is being made to host 'webstat.banque-france.fr'. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/1.26.x/advanced-usage.html#ssl-warnings
  warnings.warn(
C:\Users\admin\Anaconda\anaconda3\Lib\site-packages\urllib3\connectionpool.py:1056: InsecureRequestWarning: Unverified HTTPS request is being made to host 'webstat.banque-france.fr'. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/1.26.x/advanced-usage.html#ssl-warnings
  warnings.warn(
C:\Users\admin\Anaconda\anaconda3\Lib\site-packages\urllib3\connectionpool.py:1056: InsecureRequestWarning: Unverified HTTPS request is being made to host 'webstat.banque-france.fr'. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/1.26.x/advanced-usage.html#ssl-warnings
  warnings.warn(
C:\U

Titre_FR,Date,"Crédits de trésorerie accordés aux sociétés non financières résidentes, encours","Crédits à l'habitat accordés aux ménages et ISBLSM résidents, encours","Crédits à l'habitat accordés aux particuliers résidents, flux mensuels","Crédits à l'habitat accordés aux particuliers résidents, flux mensuels, CVS","Crédits à l'investissement accordés aux sociétés non financières résidentes, encours","Crédits à la consommation accordés aux ménages et ISBLSM résidents, encours","Dépôts à terme des SNF résidentes, encours","Dépôts à terme des ménages et ISBLSM résidents, encours","Dépôts à vue des SNF résidentes, encours","Dépôts à vue des ménages et ISBLSM résidents, encours","Livrets A des ménages et ISBLSM résidents, encours","Livrets d'épargne des ménages et ISBLSM résidents, encours","Livrets de développement durable et solidaire des ménages résidents, encours","PEL des ménages résidents, encours","Titres d'OPC monétaires, taux de valorisation de l'encours","Crédits nouveaux à l'habitat (hors renégociations) aux particuliers résidents, flux, CVS","Crédits nouveaux à l'habitat des particuliers, flux CVS","Crédits nouveaux à la consommation des ménages - hors découverts, flux CVS","Taux d'intérêt sur contrats nouveaux, DAT, SNF, durée supérieure à 2 ans"
0,2024-07-31,314237.0,1423643.0,1535.0,-1439.0,976553.0,224601.0,279410.0,431450.0,503561.0,582712.0,386625.0,909272.0,154679.0,228769.0,NaN,11300.0,13100.0,6700.0,0.035781
1,2024-06-30,314429.0,1421845.0,148.0,-1851.0,970167.0,222542.0,281423.0,430440.0,506658.0,576163.0,385013.0,906309.0,153940.0,231218.0,0.04072,8600.0,10000.0,5600.0,0.037382
2,2024-05-31,311360.0,1421534.0,-1793.0,-1694.0,967880.0,222176.0,283627.0,430023.0,497855.0,568113.0,383593.0,905220.0,153343.0,234156.0,0.04008,8100.0,9800.0,5700.0,0.038587
3,2024-04-30,311584.0,1423298.0,-1431.0,-580.0,964993.0,221823.0,282398.0,429668.0,503483.0,567811.0,382247.0,904477.0,152744.0,236746.0,0.03970,8900.0,11000.0,6500.0,0.036606
4,2024-03-31,310963.0,1424689.0,-2183.0,-1803.0,962112.0,220339.0,287702.0,429658.0,500685.0,567155.0,380563.0,902877.0,151989.0,239678.0,0.03836,6900.0,8700.0,5300.0,0.038820
5,2024-02-29,309104.0,1427062.0,-2477.0,-918.0,960184.0,219586.0,284657.0,428773.0,500414.0,565992.0,378736.0,900481.0,151092.0,243026.0,0.03734,7300.0,9800.0,6300.0,0.039753
6,2024-01-31,313441.0,1429529.0,-2450.0,-805.0,958228.0,220741.0,280305.0,428892.0,501683.0,571416.0,376482.0,898956.0,150148.0,247324.0,0.03598,7500.0,9800.0,6200.0,0.038769
7,2023-12-31,319062.0,1432120.0,98.0,-1365.0,957421.0,221376.0,276675.0,427561.0,535766.0,577732.0,373697.0,896020.0,148896.0,252902.0,0.03383,8200.0,10100.0,6200.0,0.042451
8,2023-11-30,317998.0,1432268.0,-1042.0,18.0,956317.0,221802.0,269158.0,416848.0,523906.0,578347.0,360926.0,873633.0,144087.0,250966.0,0.03189,8600.0,10800.0,6300.0,0.041784
9,2023-10-31,319295.0,1432574.0,138.0,-116.0,951718.0,221519.0,262896.0,413586.0,513494.0,583522.0,360277.0,875378.0,143869.0,254318.0,0.02963,9000.0,11300.0,6100.0,0.040664


**I.2 INSEE**

In [18]:
def insee_request(id_list):
    token = "your-insee-token"
    url = "https://bdm.insee.fr/series/sdmx/data/SERIES_BDM/"
    
    for id in id_list:
        url = url + id + "+"

    headers = {
    "Authorization": f"Bearer {token}"
    }

    params = {
    # "startPeriod": "2020-Q1",
    # "endPeriod": "2020-Q2"
    }

    response = requests.get(url, headers=headers, params=params)
    data = xmltodict.parse(response.content)
    return data

    

def parse_insee_request(data):
    df = pd.DataFrame(columns=['Date'])

    if type(data['message:StructureSpecificData']['message:DataSet']['Series']) == list:
        series = data['message:StructureSpecificData']['message:DataSet']['Series']
    else:
        series = [data['message:StructureSpecificData']['message:DataSet']['Series']]

    for serie in series:
        sub_df = pd.DataFrame()
        unit_mult = 10**float(serie['@UNIT_MULT'])
        dates = []
        values = []
        for obs in serie['Obs']:
            dates.append(obs['@TIME_PERIOD'])
            values.append(float(obs['@OBS_VALUE']) * unit_mult)
        sub_df['Date'] = dates
        
        sub_df[serie['@TITLE_FR']] = values
        df = df.merge(sub_df, on='Date', how='outer')

    if serie['@FREQ'] == 'A':
        df['Date'] = pd.to_datetime(df['Date']) + pd.offsets.MonthEnd(12)
    elif serie['@FREQ'] == 'T':
        df['Date'] = pd.to_datetime(df['Date']) + pd.offsets.MonthEnd(3)  
    elif serie['@FREQ'] == 'M':
        df['Date'] = pd.to_datetime(df['Date']) + pd.offsets.MonthEnd(1)  
    
    return df

In [19]:
annual_insee_ids = [
    '010564058',
    '010563884'
    # '010563893',
]

quarterly_insee_ids = [
    '010564925', #RDB courant
    '010565711', #Conso ménages courant
    '010565712', #Conso ménages constant
    '010564934', #Epargne ménages courant,
    '010565738', #FBCF courant ménages
    '010565734', #FBCF constant ménages
    '010565724', #Exportation courant
    '010565725', #Exportation constant
    '010565726', #Import courant
    '010565727', #Import constant
    '010565717', #Conso totale courant
    '010565718', #Conso totale constant
    '010564879', #Menages aide à l'investissement,
    '010564890',
    '010564889',
    '010564874',
    '010564983',
    '010564909',
    '010564694',
    '010564707',
    '010564734',
    '010564744',
    '010564755',
    '010564767',
    '010564766',
    '010564789',
    '010564848',
    '010564866',
    '010564865',
    '010564833',
    '010564840',
    '010564880',
    '010564892',
    '010564891',
    '010564975',
    '010564898',
    '010565707',
    '010565708',
    '010565730',
    '010565731',
    '010564901',
    '010564931',
    '010564875',
    '010564883',
    '010564884',
    '010564871',
    '010565536',
    '010565746',
    '010565745'
]

monthly_insee_ids = [
   '001769682' #CPI
]

data_insee_annual = insee_request(annual_insee_ids)
df_insee_annual = parse_insee_request(data_insee_annual)
display(df_insee_annual.head())

data_insee_quarterly = insee_request(quarterly_insee_ids)
df_insee_quarterly = parse_insee_request(data_insee_quarterly)
display(df_insee_quarterly.head())

data_insee_monthly = insee_request(monthly_insee_ids)
df_insee_monthly = parse_insee_request(data_insee_monthly)
display(df_insee_monthly.head())

,Date,Comptes nationaux annuels base 2014 - Acquisitions moins cessions d'actifs non financiers non produits - Sociétés non financières - Emplois ou actifs - Série arrêtée,Comptes nationaux annuels base 2014 - Variation de stocks - Ménages y compris EI - Emplois ou Actifs - Série arrêtée
0,2021-12-31,NaN,1.534000e+09
1,2020-12-31,522000000.0,1.534000e+09
2,2019-12-31,524000000.0,8.610000e+08
3,2018-12-31,97000000.0,1.833000e+09
4,2017-12-31,465000000.0,8.480000e+08


C:\Users\admin\AppData\Local\Temp\ipykernel_640\3602790801.py:47: UserWarning: Could not infer format, so each element will be parsed individually, falling back to `dateutil`. To ensure parsing is consistent and as-expected, please specify a format.
  df['Date'] = pd.to_datetime(df['Date']) + pd.offsets.MonthEnd(3)


,Date,Dépenses de consommation totales - Valeur aux prix courants - Série CVS-CJO - Série arrêtée,Exportations - Total - Valeur aux prix courants - Série CVS-CJO - Série arrêtée,Prestations d'assistance sociale en espèces versées par les ISBLSM - Valeur aux prix courants - Série CVS - Série arrêtée,Investissement des entreprises non financières - Total - Volume aux prix de l'année précédente chaînés - Série CVS-CJO - Série arrêtée,Variations de stocks des sociétés non financières - Valeur aux prix courants - Série CVS-CJO - Série arrêtée,Acquisitions nettes d'actifs non produits versées par les ménages (y compris les entreprises individuelles) - Valeur aux prix courants - Série CVS - Série arrêtée,Épargne des sociétés non financières - Valeur aux prix courants - Série CVS-CJO - Série arrêtée,Autres subventions sur la production reçues par les ISBLSM - Valeur aux prix courants - Série CVS - Série arrêtée,Dépenses de consommation des ménages - Total - Valeur aux prix courants - Série CVS-CJO - Série arrêtée,Masse salariale versée par les ISBLSM - Valeur aux prix courants - Série CVS - Série arrêtée,Primes nettes d'assurance-dommage versées par les ISBLSM - Valeur aux prix courants - Série CVS - Série arrêtée,Acquisitions moins cession d'objets de valeur - Valeur aux prix courants - Série CVS-CJO - Série arrêtée,Impôts en capital à payer par les sociétés non financières - Valeur aux prix courants - Série CVS - Série arrêtée,Intérêts reçus par les ISBLSM - Valeur aux prix courants - Série CVS - Série arrêtée,Impôts sur la main d'oeuvre versés par les ISBLSM - Valeur aux prix courants - Série CVS - Série arrêtée,Autres transferts en capital à payer par les ISBLSM - Valeur aux prix courants - Série CVS - Série arrêtée,FBCF des ménages - Total - Volume aux prix de l'année précédente chaînés - Série CVS-CJO - Série arrêtée,Investissement des entreprises non financières - Total - Valeur aux prix courants - Série CVS-CJO - Série arrêtée,Dépenses de consommation des ISBLSM - Total - Volume aux prix de l'année précédente chaînés - Série CVS-CJO - Série arrêtée,Transferts courants divers versés par les ISBLSM - Valeur aux prix courants - Série CVS - Série arrêtée,Dépenses de consommation totales - Volume aux prix de l'année précédente chaînés - Série CVS-CJO - Série arrêtée,Valeur ajoutée des sociétés non financières - Valeur aux prix courants - Série CVS-CJO - Série arrêtée,Autres transferts en capital reçus par les sociétés non financières - Valeur aux prix courants - Série CVS - Série arrêtée,Valeur ajoutée des ISBLSM - Valeur aux prix courants - Série CVS-CJO - Série arrêtée,Autres transferts en capital à payer par les sociétés non financières - Valeur aux prix courants - Série CVS - Série arrêtée,Aides à l'investissement reçues par les ménages (y compris les entreprises individuelles) - Valeur aux prix courants - Série CVS - Série arrêtée,Importations - Total - Valeur aux prix courants - Série CVS-CJO - Série arrêtée,Transferts courants divers reçus par les ISBLSM - Valeur aux prix courants - Série CVS - Série arrêtée,Autres transferts en capital reçus par les ISBLSM - Valeur aux prix courants - Série CVS - Série arrêtée,Importations - Total - Volume aux prix de l'année précédente chaînés - Série CVS-CJO - Série arrêtée,Exportations - Total - Volume aux prix de l'année précédente chaînés - Série CVS-CJO - Série arrêtée,Impôts en capital à payer par les ménages (y compris les entreprises individuelles) - Valeur aux prix courants - Série CVS - Série arrêtée,Revenu disponible brut des ménages (y compris les entreprises individuelles) - Valeur aux prix courants - Série CVS-CJO - Série arrêtée,Aides à l'investissement reçues par les ISBLSM - Valeur aux prix courants - Série CVS - Série arrêtée,Épargne des ménages (y compris les entreprises individuelles) - Valeur aux prix courants - Série CVS-CJO - Série arrêtée,Autres impôts sur la production versés par les ISBLSM - Valeur aux prix courants - Série CVS - Série arrêtée,Produit intérieur br

,Date,Indice CVS des prix à la consommation - Base 2015 - Ensemble des ménages - France - Ensemble
0,2024-07-31,120.22
1,2024-06-30,119.76
2,2024-05-31,119.71
3,2024-04-30,119.73
4,2024-03-31,119.10


# I. SNF

In [20]:
train_range = ('2003-03-01', '2022-08-01')

## 1. Arbitrage

In [11]:
#scenario central

In [21]:
df_scenario = pd.read_excel('20230915 - RUN/Envoi_Data_Scenario_France_Central_APR_20230918.xlsx')
df_scenario['Date'] = pd.to_datetime(df_scenario['Date'])
df_scenario.head()

,Unnamed: 0,Date,OAT_10Y,EUSW10V3,GDP_courant,GDP_constant,Euribor_3M,CPI,Inflation,CAC40,LA_rate,RDB_constant,RDB_constant (monthly transformed),RDB_courant,RDB_courant (monthly transformed),taux_chomage,"GDP / current prices, % ch yoy / quarterly","GDP / constant prices, % ch yoy / quarterly","Households disposable income / constant prices (deflated by household consumption deflator), % ch yoy / quarterly","Household consumption / constant prices, % ch yoy / quarterly","Public consumption / constant prices, % ch yoy / quarterly","Investment, private, households / constant prices, % ch yoy / quarterly","Imports of goods and services / constant prices, % ch yoy / quarterly","Exports of goods and services / constant prices, % ch yoy / quarterly",CPI index / % ch yoy / quarterly,"Investment, private, NFC / constant prices, % ch yoy / quarterly","Non financial companies' saving ratio / constant prices, % of value added / quarterly","Non financial companies' profit ratio / constant prices, % of value added / quarterly"
0,0,2000-01-31,5.667,5.9090,120586.469136,149927.037037,3.34314,77.4,1.54,5661.53,2.25,0.0,0.0,0.0,71971.395062,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.00000,0.000000,0.000000,0.000000,0.000000,0.000000
1,1,2000-02-29,5.626,5.9210,120783.617284,150098.259259,3.53675,77.5,1.36,6121.95,2.25,0.0,0.0,0.0,72106.098765,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.00000,0.000000,0.000000,0.000000,0.000000,0.000000
2,2,2000-03-31,5.452,5.7735,121157.604938,150428.345679,3.74704,77.9,1.49,6374.75,2.25,0.0,0.0,216466.0,72376.382716,8.7,5.243168,4.357053,2.961124,4.776229,2.050529,5.579869,13.37296,12.360544,1.570133,6.888763,21.754064,32.367585
3,3,2000-04-30,5.356,5.7127,122064.148148,151213.679012,3.92904,77.9,1.21,6228.65,2.25,0.0,0.0,0.0,73008.197531,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.00000,0.000000,0.000000,0.000000,0.000000,0.000000
4,4,2000-05-31,5.501,5.9057,122665.259259,151695.975309,4.35038,78.1,1.40,6346.45,2.25,0.0,0.0,0.0,73363.271605,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.00000,0.000000,0.000000,0.000000,0.000000,0.000000


In [22]:
""" Unique processing """
df_scenario['RDB_constant'] = df_scenario['RDB_constant (monthly transformed)']
df_scenario['RDB_courant'] = df_scenario['RDB_courant (monthly transformed)']
df_scenario["GDP / current prices, % ch yoy / quarterly"] = df_scenario["GDP / current prices, % ch yoy / quarterly"].replace(0, np.nan).bfill() / 100
df_scenario["GDP / constant prices, % ch yoy / quarterly"] = df_scenario["GDP / constant prices, % ch yoy / quarterly"].replace(0, np.nan).bfill() / 100
df_scenario["Households disposable income / constant prices (deflated by household consumption deflator), % ch yoy / quarterly"] = df_scenario["Households disposable income / constant prices (deflated by household consumption deflator), % ch yoy / quarterly"].replace(0, np.nan).bfill() / 100
df_scenario["Household consumption / constant prices, % ch yoy / quarterly"] = df_scenario["Household consumption / constant prices, % ch yoy / quarterly"].replace(0, np.nan).bfill() / 100
df_scenario["Public consumption / constant prices, % ch yoy / quarterly"] = df_scenario["Public consumption / constant prices, % ch yoy / quarterly"].replace(0, np.nan).bfill() / 100
df_scenario["Investment, private, households / constant prices, % ch yoy / quarterly"] = df_scenario["Investment, private, households / constant prices, % ch yoy / quarterly"].replace(0, np.nan).bfill() / 100
df_scenario["Imports of goods and services / constant prices, % ch yoy / quarterly"] = df_scenario["Imports of goods and services / constant prices, % ch yoy / quarterly"].replace(0, np.nan).bfill() / 100
df_scenario["Exports of goods and services / constant prices, % ch yoy / quarterly"] = df_scenario["Exports of goods and services / constant prices, % ch yoy / quarterly"].replace(0, np.nan).bfill() / 100
df_scenario["CPI index / % ch yoy / quarterly"] = df_scenario["CPI index / % ch yoy / quarterly"].replace(0, np.nan).bfill() / 100
df_scenario["Investment, private, NFC / constant prices, % ch yoy / quarterly"] = df_scenario["Investment, private, NFC / constant prices, % ch yoy / quarterly"].replace(0, np.nan).bfill() / 100
df_scenario["Non financial companies' saving ratio / constant prices, % of value added / quarterly"] = df_scenario["Non financial companies' saving ratio / constant prices, % of value added / quarterly"].replace(0, np.nan).bfill() / 100
df_scenario["Non financial companies' profit ratio / constant prices, % of value added / quarterly"] = df_scenario["Non financial companies' profit ratio / constant prices, % of value added / quarterly"].replace(0, np.nan).bfill() / 100

pd.options.display.max_columns = None
df_scenario.head()

,Unnamed: 0,Date,OAT_10Y,EUSW10V3,GDP_courant,GDP_constant,Euribor_3M,CPI,Inflation,CAC40,LA_rate,RDB_constant,RDB_constant (monthly transformed),RDB_courant,RDB_courant (monthly transformed),taux_chomage,"GDP / current prices, % ch yoy / quarterly","GDP / constant prices, % ch yoy / quarterly","Households disposable income / constant prices (deflated by household consumption deflator), % ch yoy / quarterly","Household consumption / constant prices, % ch yoy / quarterly","Public consumption / constant prices, % ch yoy / quarterly","Investment, private, households / constant prices, % ch yoy / quarterly","Imports of goods and services / constant prices, % ch yoy / quarterly","Exports of goods and services / constant prices, % ch yoy / quarterly",CPI index / % ch yoy / quarterly,"Investment, private, NFC / constant prices, % ch yoy / quarterly","Non financial companies' saving ratio / constant prices, % of value added / quarterly","Non financial companies' profit ratio / constant prices, % of value added / quarterly"
0,0,2000-01-31,5.667,5.9090,120586.469136,149927.037037,3.34314,77.4,1.54,5661.53,2.25,0.0,0.0,71971.395062,71971.395062,0.0,0.052432,0.043571,0.029611,0.047762,0.020505,0.055799,0.133730,0.123605,0.015701,0.068888,0.217541,0.323676
1,1,2000-02-29,5.626,5.9210,120783.617284,150098.259259,3.53675,77.5,1.36,6121.95,2.25,0.0,0.0,72106.098765,72106.098765,0.0,0.052432,0.043571,0.029611,0.047762,0.020505,0.055799,0.133730,0.123605,0.015701,0.068888,0.217541,0.323676
2,2,2000-03-31,5.452,5.7735,121157.604938,150428.345679,3.74704,77.9,1.49,6374.75,2.25,0.0,0.0,72376.382716,72376.382716,8.7,0.052432,0.043571,0.029611,0.047762,0.020505,0.055799,0.133730,0.123605,0.015701,0.068888,0.217541,0.323676
3,3,2000-04-30,5.356,5.7127,122064.148148,151213.679012,3.92904,77.9,1.21,6228.65,2.25,0.0,0.0,73008.197531,73008.197531,0.0,0.060883,0.047734,0.033617,0.048541,0.021410,0.045539,0.166557,0.149021,0.015385,0.061023,0.217187,0.328213
4,4,2000-05-31,5.501,5.9057,122665.259259,151695.975309,4.35038,78.1,1.40,6346.45,2.25,0.0,0.0,73363.271605,73363.271605,0.0,0.060883,0.047734,0.033617,0.048541,0.021410,0.045539,0.166557,0.149021,0.015385,0.061023,0.217187,0.328213


In [23]:
df_bdf_quarterly

Titre_FR,Date,Actif Toutes dénominations monétaires des SNF vis-à-vis de toutes contreparties - encours d'OPC monétaires,Actif Toutes dénominations monétaires des SNF vis-à-vis de toutes contreparties - flux d'actions et titres d'OPC,Actif Toutes dénominations monétaires des SNF vis-à-vis de toutes contreparties - flux de police d'assurance et fonds de pension,Actif Toutes dénominations monétaires des SNF vis-à-vis de toutes contreparties - flux de titres de créances,Actif Toutes dénominations monétaires des ménages vis-à-vis de toutes contreparties - encours de monnaie et dépôts,Actif Toutes dénominations monétaires des ménages vis-à-vis de toutes contreparties - flux d'actions et titres d'OPC,Actif Toutes dénominations monétaires des ménages vis-à-vis de toutes contreparties - flux de crédits,Actif Toutes dénominations monétaires des ménages vis-à-vis de toutes contreparties - flux de titres de créances,Actif Toutes dénominations monétaires des ménages vis-à-vis de toutes contreparties - flux des autres comptes à recevoir ou à payer,"Assurance-vie en unités de compte, actif des ménages, encours trimestriel","Assurance-vie support euro, actif des ménages, encours trimestriel","Ménages et ISBLSM, Actif, Monnaie et dépôts",Passif Toutes dénominations monétaires des SNF vis-à-vis de toutes contreparties - flux d'actions et titres d'OPC,Passif Toutes dénominations monétaires des SNF vis-à-vis de toutes contreparties - flux de titres de créances,Passif Toutes dénominations monétaires des ménages vis-à-vis de toutes contreparties - flux d'actions et titres d'OPC,Passif Toutes dénominations monétaires des ménages vis-à-vis de toutes contreparties - flux de crédits,Passif Toutes dénominations monétaires des ménages vis-à-vis de toutes contreparties - flux des autres comptes à recevoir ou à payer,Passif des SNF vis-à-vis de toutes contreparties - flux de police d'assurance et fonds de pension,"Sociétés non financières, Actif, Autres comptes à recevoir / à payer","Sociétés non financières, Passif, Autres comptes à recevoir / à payer"
185,2024-03-31,116555000.0,-15821000.0,4065000.0,4625000.0,1.987605e+09,-5019000.0,93000.0,1658000.0,13723000.0,516600.0,1498488.0,2.083687e+09,23514000.0,-4120000.0,0.0,-5268000.0,-4908000.0,0.0,1.670124e+09,1.275308e+09
184,2023-12-31,95009000.0,51784000.0,-601000.0,-1991000.0,1.986409e+09,-1270000.0,1081000.0,1527000.0,-26891000.0,489101.0,1482693.0,2.082127e+09,44831000.0,4961000.0,0.0,1822000.0,1423000.0,0.0,1.581359e+09,1.260307e+09
183,2023-09-30,96868000.0,23079000.0,-978000.0,4094000.0,1.991095e+09,8511000.0,-791000.0,1927000.0,16049000.0,465300.0,1427255.0,2.084891e+09,20833000.0,6967000.0,0.0,3903000.0,-1458000.0,0.0,1.534746e+09,1.242780e+09
182,2023-06-30,89122000.0,37304000.0,217000.0,2886000.0,1.971170e+09,9718000.0,500000.0,1818000.0,20473000.0,472500.0,1453955.0,2.065474e+09,27373000.0,2721000.0,0.0,9682000.0,-16179000.0,0.0,1.528596e+09,1.209753e+09
181,2023-03-31,93754000.0,-18786000.0,3686000.0,2083000.0,1.952205e+09,1219000.0,-12000.0,577000.0,27610000.0,457000.0,1460436.0,2.046393e+09,9858000.0,-3476000.0,0.0,7872000.0,18326000.0,0.0,1.517714e+09,1.191769e+09
180,2022-12-31,75421000.0,17331000.0,-1617000.0,4975000.0,1.932922e+09,7316000.0,-977000.0,1382000.0,-30768000.0,432500.0,1451084.0,2.027017e+09,32964000.0,1248000.0,0.0,15422000.0,-13483000.0,0.0,1.507163e+09,1.189609e+09
179,2022-09-30,65343000.0,35016000.0,-165000.0,1218000.0,1.933260e+09,5741000.0,32000.0,2604000.0,3061000.0,440700.0,1435705.0,2.026928e+09,16868000.0,-2362000.0,0.0,22379000.0,-12255000.0,0.0,1.465127e+09,1.169398e+09
178,2022-06-30,62477000.0,-8690000.0,-1124000.0,7423000.0,1.900555e+09,24104000.0,675000.0,2418000.0,-237000.0,447699.0,1492032.0,1.995886e+09,27737000.0,-4190000.0,0.0,29823000.0,-10820000.0,0.0,1.443986e+09,1.133223e+09
177,2022-03-31,65619000.0,47302000.0,2588000.0,3031000.0,1.871204e+09,-943000.0,485000.0,-258000.0,48742000.0,473200.0,1598413.0,1.966340e+09,13775000.0,707000.0,0.0,16159000.0,

In [24]:
df_target = df_bdf_monthly[['Date', 'Dépôts à vue des SNF résidentes, encours', 'Dépôts à terme des SNF résidentes, encours']]
df_target = df_target.merge(df_bdf_quarterly[["Date", "Actif Toutes dénominations monétaires des SNF vis-à-vis de toutes contreparties - encours d'OPC monétaires"]], on='Date', how='left')
df_target["Actif Toutes dénominations monétaires des SNF vis-à-vis de toutes contreparties - encours d'OPC monétaires"] = df_target["Actif Toutes dénominations monétaires des SNF vis-à-vis de toutes contreparties - encours d'OPC monétaires"].bfill() / 1e3

In [25]:
df_target #valeur en mensuelles

Titre_FR,Date,"Dépôts à vue des SNF résidentes, encours","Dépôts à terme des SNF résidentes, encours",Actif Toutes dénominations monétaires des SNF vis-à-vis de toutes contreparties - encours d'OPC monétaires
0,2024-07-31,503561.0,279410.0,116555.0
1,2024-06-30,506658.0,281423.0,116555.0
2,2024-05-31,497855.0,283627.0,116555.0
3,2024-04-30,503483.0,282398.0,116555.0
4,2024-03-31,500685.0,287702.0,116555.0
5,2024-02-29,500414.0,284657.0,95009.0
6,2024-01-31,501683.0,280305.0,95009.0
7,2023-12-31,535766.0,276675.0,95009.0
8,2023-11-30,523906.0,269158.0,96868.0
9,2023-10-31,513494.0,262896.0,96868.0


In [31]:
### SNF
model_dav_part_stable_SNF = load(f'models/dav_part_stable_SNF.joblib')
drivers = ['OAT_10Y_diff', 'EUSW10V3_diff', 'Inflation_diff', 'SNF_flux_credit'] + [f"month_{i}" for i in range(3, 13)]

In [34]:
df_dav_SNF_histo = df_bdf_monthly[['Date', "Crédits à l'investissement accordés aux sociétés non financières résidentes, encours", "Crédits de trésorerie accordés aux sociétés non financières résidentes, encours", "Dépôts à vue des SNF résidentes, encours"]]
df_dav_SNF_histo = df_dav_SNF_histo.merge(df_scenario, on='Date', how='left')
df_dav_SNF_histo['SNF_flux_credit'] = df_dav_SNF_histo[["Crédits à l'investissement accordés aux sociétés non financières résidentes, encours", "Crédits de trésorerie accordés aux sociétés non financières résidentes, encours"]].sum(axis=1).diff()
df_dav_SNF_histo['OAT_10Y_diff'] = df_dav_SNF_histo['OAT_10Y'].diff()
df_dav_SNF_histo['EUSW10V3_diff'] = df_dav_SNF_histo['EUSW10V3'].diff()
df_dav_SNF_histo['Inflation_diff'] = df_dav_SNF_histo['Inflation'].diff()
for i in range(1,13):
    df_dav_SNF_histo[f"month_{i}"] = (df_dav_SNF_histo['Date'].dt.month == i).astype(int)
df_dav_SNF_histo = df_dav_SNF_histo[df_dav_SNF_histo['Date'] >= "2013-12"]
df_dav_SNF_histo = df_dav_SNF_histo.sort_values('Date', ascending=True)

display(df_dav_SNF_histo)
#on obtient l'historique des différences des différents drivers avec l'indication du mois 

#histo jusqu'en 07/24

,Date,"Crédits à l'investissement accordés aux sociétés non financières résidentes, encours","Crédits de trésorerie accordés aux sociétés non financières résidentes, encours","Dépôts à vue des SNF résidentes, encours",Unnamed: 0,OAT_10Y,EUSW10V3,GDP_courant,GDP_constant,Euribor_3M,CPI,Inflation,CAC40,LA_rate,RDB_constant,RDB_constant (monthly transformed),RDB_courant,RDB_courant (monthly transformed),taux_chomage,"GDP / current prices, % ch yoy / quarterly","GDP / constant prices, % ch yoy / quarterly","Households disposable income / constant prices (deflated by household consumption deflator), % ch yoy / quarterly","Household consumption / constant prices, % ch yoy / quarterly","Public consumption / constant prices, % ch yoy / quarterly","Investment, private, households / constant prices, % ch yoy / quarterly","Imports of goods and services / constant prices, % ch yoy / quarterly","Exports of goods and services / constant prices, % ch yoy / quarterly",CPI index / % ch yoy / quarterly,"Investment, private, NFC / constant prices, % ch yoy / quarterly","Non financial companies' saving ratio / constant prices, % of value added / quarterly","Non financial companies' profit ratio / constant prices, % of value added / quarterly",SNF_flux_credit,OAT_10Y_diff,EUSW10V3_diff,Inflation_diff,month_1,month_2,month_3,month_4,month_5,month_6,month_7,month_8,month_9,month_10,month_11,month_12
127,2013-12-31,567952.0,167476.0,230949.0,167.0,2.332000,2.084400,178250.925926,179060.666667,0.275360,99.730000,0.710000,4166.62,1.25,112382.793030,112382.793030,108324.790123,108324.790123,9.800000,0.019496,0.014097,-0.003938,0.008372,0.015630,0.021622,0.046238,0.033884,0.007834,0.021074,0.212288,0.299254,-2909.0,-0.055000,0.027400,0.060000,0,0,0,0,0,0,0,0,0,0,0,1
126,2014-01-31,569032.0,169305.0,213521.0,168.0,2.387000,2.057000,178308.259259,178778.197531,0.291830,99.090000,0.650000,4250.47,1.25,112822.699131,112822.699131,108880.469136,108880.469136,0.000000,0.018564,0.013161,0.007400,0.004023,0.014614,0.003867,0.053897,0.044738,0.008602,0.021133,0.205187,0.301244,80.0,0.134000,0.172700,-0.270000,1,0,0,0,0,0,0,0,0,0,0,0
125,2014-02-28,570815.0,167442.0,215802.0,169.0,2.253000,1.884300,178439.592593,178773.827160,0.288100,99.700000,0.920000,4300.84,1.25,113039.941899,113039.941899,109126.839506,109126.839506,0.000000,0.018564,0.013161,0.007400,0.004023,0.014614,0.003867,0.053897,0.044738,0.008602,0.021133,0.205187,0.301244,456.0,0.102000,0.055600,0.310000,0,1,0,0,0,0,0,0,0,0,0,0
124,2014-03-31,570465.0,167336.0,218558.0,170.0,2.151000,1.828700,178547.148148,178797.975309,0.305330,100.210000,0.610000,4338.05,1.25,113202.054899,113202.054899,109308.691358,109308.691358,9.800000,0.018564,0.013161,0.007400,0.004023,0.014614,0.003867,0.053897,0.044738,0.008602,0.021133,0.205187,0.301244,-3632.0,0.129000,0.072500,-0.110000,0,0,1,0,0,0,0,0,0,0,0,0
123,2014-04-30,569314.0,172119.0,219517.0,171.0,2.022000,1.756200,178469.098765,178764.567901,0.329590,100.180000,0.720000,4436.79,1.25,113239.232478,113239.232478,109307.555556,109307.555556,0.000000,0.011028,0.006885,0.008363,0.007199,0.011918,-0.018754,0.040862,0.017329,0.007509,0.017204,0.208521,0.301200,-1710.0,0.174000,0.126800,0.040000,0,0,0,1,0,0,0,0,0,0,0,0
122,2014-05-31,570590.0,172553.0,221168.0,172.0,1.848000,1.629400,178650.469136,178910.308642,0.325230,100.210000,0.680000,4484.96,1.25,113343.440179,113343.440179,109449.222222,109449.222222,0.000000,0.011028,0.006885,0.008363,0.007199,0.011918,-0.018754,0.040862,0.017329,0.007509,0.017204,0.208521,0.301200,-2927.0,0.134000,0.098000,0.200000,0,0,0,0,1,0,0,0,0,0,0,0
121,2014-06-30,570658.0,175412.0,230489.0,173.0,1.714000,1.531400,178929.432099,179149.123457,0.241430,100.160000,0.480000,4522.01,1.25,113444.872351,113444.872351,109615.222222,109615.222222,9.800000,0.011028,0.006885,0.008363,0.007199,0.011918,-0.018754,0.040862,0.017329,0.007509,0.017204,0.208521,0.301200,-2215.0,0.149000,0.132900,0.020000,0,0,0,0,0,1,0,0,0,0,0,0
120,2014-07-31,572620.0,1

In [ ]:
#Application du modèle et projection pour la partie stable du DAV

df_dav_SNF_histo['dav_part_stable_SNF_diff'] = model_dav_part_stable_SNF.predict(df_dav_SNF_histo[drivers].fillna(0))
dav_part_stable_init =  df_dav_SNF_histo[(df_dav_SNF_histo['Date'].dt.year==2013) & (df_dav_SNF_histo['Date'].dt.month==12)]['Dépôts à vue des SNF résidentes, encours'].values[0] 
df_dav_SNF_histo['dav_part_stable_SNF_histo'] = dav_part_stable_init + (df_dav_SNF_histo['dav_part_stable_SNF_diff'] * (df_dav_SNF_histo['Date'].dt.year > 2013)).cumsum()

df_target = df_target.merge(df_dav_SNF_histo[['Date', 'dav_part_stable_SNF_histo']], on='Date', how='left')


In [30]:
df_target

,Date,"Dépôts à vue des SNF résidentes, encours","Dépôts à terme des SNF résidentes, encours",Actif Toutes dénominations monétaires des SNF vis-à-vis de toutes contreparties - encours d'OPC monétaires,dav_part_stable_SNF_histo
0,2024-07-31,503561.0,279410.0,116555.0,410471.699787
1,2024-06-30,506658.0,281423.0,116555.0,409014.694787
2,2024-05-31,497855.0,283627.0,116555.0,404026.338784
3,2024-04-30,503483.0,282398.0,116555.0,401248.220564
4,2024-03-31,500685.0,287702.0,116555.0,399716.794211
5,2024-02-29,500414.0,284657.0,95009.0,398417.697963
6,2024-01-31,501683.0,280305.0,95009.0,399630.866661
7,2023-12-31,535766.0,276675.0,95009.0,401314.942051
8,2023-11-30,523906.0,269158.0,96868.0,391388.356290
9,2023-10-31,513494.0,262896.0,96868.0,389489.137815


In [35]:
dav_part_stable_init

230949.0